In [1]:
import modal
import pandas as pd
import evaluate


# load my modal function from deployment to do inference

def run_inference(d):
    # From Modal, get inference of datapoint d and return

    # grabs function from Modal to run my large language model
    extract = modal.Function.lookup("pkmn-py", "run_inference")
    # call run_inference remotely on modal
    result = extract.call(d)
    return result


/Users/ArjunPatel/opt/anaconda3/envs/pkmn-moves/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-07 19:45:13.822800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
rouge = evaluate.load("rouge")

In [5]:
# import my evaluation jsonl file so we can run inference on it 


# import evaluation jsonl speed_training_data_eval.jsonl

import jsonlines


eval_set = []
f = "speed_training_data_eval.jsonl"
with jsonlines.open(f) as reader:
        for entry in reader:
            eval_set.append(entry)
            

In [11]:
eval_set = pd.DataFrame(eval_set)

In [12]:
eval_set

,prompt,output
0,Is -5 35 Noibat slower than -5 19 Blissey?,"\n{\n ""p1"": ""Noibat"",\n ""p2"": ""Blissey"",..."
1,Does -6 32 Larvesta outspeed -5 105 Arrokuda?,"\n{\n ""p1"": ""Larvesta"",\n ""p2"": ""Arrokud..."
2,Check if +2 235 Great Tusk outspeeds -3 133 Al...,"\n{\n ""p1"": ""Great Tusk"",\n ""p2"": ""Altar..."
3,Will +3 with 2 speed Pawmot underspeed...,"\n{\n ""p1"": ""Pawmot"",\n ""p2"": ""Ursaring""..."
4,Will Psyduck at 0 with 110 outspeed Pawmot at ...,"\n{\n ""p1"": ""Psyduck"",\n ""p2"": ""Pawmot"",..."
...,...,...
195,Check if Greedent with 161 speed evs outspeeds...,"\n{\n ""p1"": ""Greedent"",\n ""p2"": ""Pelippe..."
196,Calculate if Marill with 149 speed evs outspee...,"\n{\n ""p1"": ""Marill"",\n ""p2"": ""Mismagius..."
197,Check if Gholdengo with 86 speed evs outspeeds...,"\n{\n ""p1"": ""Gholdengo"",\n ""p2"": ""Glimmo..."
198,Calculate if Hatterene with 221 speed evs outs...,"\n{\n ""p1"": ""Hatterene"",\n ""p2"": ""Great ..."


In [13]:
# now run inference on dataframe eval set, and store result in new column


eval_set["inference"] = eval_set["prompt"].apply(run_inference)

In [16]:
eval_set

,prompt,output,inference
0,Is -5 35 Noibat slower than -5 19 Blissey?,"\n{\n ""p1"": ""Noibat"",\n ""p2"": ""Blissey"",...","{\n ""p1"": ""Noibat"",\n ""p2"": ""Blissey"",\n..."
1,Does -6 32 Larvesta outspeed -5 105 Arrokuda?,"\n{\n ""p1"": ""Larvesta"",\n ""p2"": ""Arrokud...","{\n ""p1"": ""Larvesta"",\n ""p2"": ""Arrokuda""..."
2,Check if +2 235 Great Tusk outspeeds -3 133 Al...,"\n{\n ""p1"": ""Great Tusk"",\n ""p2"": ""Altar...","{\n ""p1"": ""Great Tusk"",\n ""p2"": ""Altaria..."
3,Will +3 with 2 speed Pawmot underspeed...,"\n{\n ""p1"": ""Pawmot"",\n ""p2"": ""Ursaring""...","{\n ""p1"": ""Pawmot"",\n ""p2"": ""Ursaring"",\..."
4,Will Psyduck at 0 with 110 outspeed Pawmot at ...,"\n{\n ""p1"": ""Psyduck"",\n ""p2"": ""Pawmot"",...","{\n ""p1"": ""Psyduck"",\n ""p2"": ""Pawmot"",\n..."
...,...,...,...
195,Check if Greedent with 161 speed evs outspeeds...,"\n{\n ""p1"": ""Greedent"",\n ""p2"": ""Pelippe...","{\n ""p1"": ""Greedent"",\n ""p2"": ""Pelipper""..."
196,Calculate if Marill with 149 speed evs outspee...,"\n{\n ""p1"": ""Marill"",\n ""p2"": ""Mismagius...","{\n ""p1"": ""marill"",\n ""p2"": ""mismagius"",..."
197,Check if Gholdengo with 86 speed evs outspeeds...,"\n{\n ""p1"": ""Gholdengo"",\n ""p2"": ""Glimmo...","{\n ""p1"": ""Gholdengo"",\n ""p2"": ""Glimmora..."
198,Calculate if Hatterene with 221 speed evs outs...,"\n{\n ""p1"": ""Hatterene"",\n ""p2"": ""Great ...","{\n ""p1"": ""Hatterene"",\n ""p2"": ""Great Tu..."


In [22]:
# compute rouge score row by row using inference columna and output column

score_result = rouge.compute(predictions = eval_set["inference"], references = eval_set["output"])


# compute the mean, median, rouge score for the eval set

# pretty print both scores
score_result


{'rouge1': 0.9991666666666669,
 'rouge2': 0.998235294117647,
 'rougeL': 0.9991666666666668,
 'rougeLsum': 0.9991666666666668}

In [23]:
score_result = rouge.compute(predictions = eval_set["prompt"], references = eval_set["output"])


In [24]:
score_result

{'rouge1': 0.3090064765206687,
 'rouge2': 0.008736770413721517,
 'rougeL': 0.19973229832874284,
 'rougeLsum': 0.3070525841896002}